In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
#from eunjeon import Mecab
from konlpy.tag import Mecab
from hanspell import spell_checker

E_list = [
    
    'EF', 'EP', 'VCP', 'VCN', 'XSA', 'XSV', 'NP', 'JX', 'JKB', #이는 어말 처리를 위한 것들이다. 어말 부분에 추가할 것이 있다면 여기에
                                                              
]

##높임말 리스트. 다음에 해당하면 높임말로 판단한다.
H_LIST = ['니다', '니까', '요', '시오', '죠', '오', '시다']

H_LIST_jam = ['ㄴㅣㄷㅏ', 'ㄴㅣㄲㅏ', 'ㅇㅛ', 'ㅅㅣㅇㅗ', 'ㅈㅛ', 'ㅇㅗ', 'ㅅㅣㄷㅏ']

P_LIST = ['.', '?', '!', '\"', '\'']

con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

lis_beta = ['EP+EF', 'VCP+EF', 'VCP', 'B+EF', 'B+EP+EF', 'B+VCP+EF', 'EF','EP']

lis_beta_ef = ['EP+EF', 'VCP+EF', 'EF']

lis_end = [
    'ㄷㅏ',
    'ㅂㄴㅣㄷㅏ',
    'ㅅㅔㅇㅛ', 'ㄷㅔㅇㅛ', 'ㅇㅔㅇㅛ', 'ㅇㅖㅇㅛ', 'ㄴㅏㅇㅛ', 'ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ','ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㅂㄴㅣㄲㅏ', 'ㄴㅣㄲㅏ', 'ㄲㅏ', 
    
]

lis_end_2low = [
    
    'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄷㅏ', 
    'ㄹㅏ',
    'ㅇㅑ', 
    
]

lis_plus = [
    
    'EP', 'VCP', 
    
]

jongsung_list = [ 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

mec = Mecab()

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

def rememberSpace(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1
        
def union_t(lis_lis):
    
    for i in range(len(lis_lis)-1):
        if lis_lis[i+1] != ' ' and lis_lis[i+1] !='SF' and lis_lis[i] != ' ' and lis_lis[i] !='SF':
            lis_lis[i] = lis_lis[i]+'/'
        if lis_lis[i+1]=='SF':
            lis_lis[i+1] = ' '+lis_lis[i+1]
            
def union_w(lis_w, lis_tag):
    
    for i in range(len(lis_w)-1):
        if lis_tag[i+1]==' SF':
            lis_w[i+1] = ' '+lis_w[i+1]
            
def prepro(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t(lis_t)
    union_w(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)-1):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1
        
        #element = element[:-1]
        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
    
    return data_w, data_t, lis_ind

def pre_target(input, lis, target):
    result = input
    if target in input:
        res = input.split('/')
        for j in range(len(res)):
            if target in res[j] and '+' in res[j]:
                loc = res[j].index('+')
                wd = res[j][:loc]
                if wd not in lis:
                    res[j] = re.sub(wd, 'A', res[j])
        result = '/'.join(res)
    return result

def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result

def isException(input):
    if 'special' in input[1]:
        return 1
    else:
        return 0
    
def delete_end(w, t, list_tag, list_end):
    
    lis_w = []
    lis_t = []
    number = len(w)
    print(len(w))
    print(len(t))
    for i in range(len(w)):
        ele = pre_target_b(t[i], lis_beta, 'EF')
        res1 = ''
        res2 = ''
        if ele not in list_tag:
            res1 = w[i]
            res2 = t[i]
        elif ele=='B+EF':
            flag = 0
            for j in list_end:
                if j in w[i]:
                    flag=1
                    ind = w[i].index(j)
                    res1 = w[i][:ind]
                    res2 = t[i]
            if flag==0:
                res1 = w[i]
                res2 = t[i]
        lis_w.append(res1)
        lis_t.append(res2)
        number = number-1
    return lis_w, lis_t

def prepro_b(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t(lis_t)
    union_w(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    jam3 = Jamodealer(lis_tag)
    
    number, jam1.jamo, jam3.jamo = delete_end(jam1.jamo, jam3.jamo, lis_beta, lis_end)
    
    
    for i in range(len(data_w)-number-1):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1
        
        #element = element[:-1]
        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
    
    return data_w, data_t, lis_ind

def pre_target_b(input, lis, target):
    result = input
    if target in input:
        res = input.split('/')
        for j in range(len(res)):
            if target in res[j] and '+' in res[j]:
                loc = res[j].index('+')
                wd = res[j][:loc]
                if wd not in lis:
                    res[j] = re.sub(wd, 'B', res[j])
        result = '/'.join(res)
    return result

def pre_target_b_test(input, lis, target):
    result = input
    if target in input:
        res = input.split('/')
        for j in range(len(res)):
            flag=0
            for k in lis:
                if k in res[j] and len(res[j].replace(target, '', 1))!=0 and flag==0:
                    loc = res[j].index(k)
                    #wd = res[j][:loc]
                    res[j] = res[j].replace(res[j][:loc], 'B+', 1)
                    flag=1
                
        result = '/'.join(res)
    return result

def delete_end_only(w, t, list_tag, list_end):
    
    lis_w = []
    lis_t = []
    
    w_last = w[-1]
    t_last = t[-1]
    
    number = len(w)
    for i in range(len(w)-1):
        if t[i+1]=='SF' or t[i+1]=='SV':
            ele = pre_target_b_test(t[i], lis_beta, 'EF')
            res1 = ''
            res2 = ''
            if ele not in list_tag:
                res1 = w[i]
                res2 = t[i]
            elif ele=='B+EF':
                flag = 0
                for j in list_end:
                    if j in w[i]:
                        flag=1
                        ind = w[i].index(j)
                        res1 = w[i][:ind]
                        res2 = t[i]
                if flag==0:
                    res1 = w[i]
                    res2 = t[i]
            lis_w.append(res1)
            lis_t.append(res2)
        else:
            lis_w.append(w[i])
            lis_t.append(t[i])
        number = number-1
    
    lis_w[-1] = w_last
    lis_t[-1] = t_last
    
    return lis_w, lis_t

def delete_end_only_test(w, t, list_tag, list_end):
    
    lis_w = []
    lis_t = []
    #print(lis_)
    w_last = w[-1]
    t_last = t[-1]
    
    last_w = []
    
    number = len(w)
    for i in range(len(w)-1):
        if t[i+1]=='SF':
            ele = pre_target_b_test(t[i], lis_beta, 'EF')
            res1 = ''
            res2 = ''
            if ele not in list_tag:
                res1 = w[i]
                res2 = t[i]
                
            elif 'B+' not in ele:
                last_w.append(w[i])
                
            else:
                #print('ee')
                flag = 0
                for j in list_end:
                    if j in w[i] and flag==0:
                        flag=1
                        ind = w[i].index(j)
                        #res1 = w[i][:ind]
                        last_w.append(j)
#                         print(j)
                        res1 = w[i].replace(j, '', 1)
                        res2 = t[i]
                        flag_plus = 0
                        for k in lis_plus:
                            if k in res2 and flag_plus==0:
                                res2 = res2.replace(k, '', 1)
                                flag_plus=1
                                
                        #res2 = t[i]
                        res2 = res2.replace('+EF', '', 1)
                if flag==0:
                    res1 = w[i]
                    res2 = t[i]
                
            lis_w.append(res1)
            #print(lis_w)
            #print(res1)
            lis_t.append(res2)
            #print(res2)
        else:
            lis_w.append(w[i])
            lis_t.append(t[i])
        number = number-1
    
    lis_w.append(w_last)
    lis_t.append(t_last)
    
    return lis_w, lis_t, last_w

def to2lists_beta(input, lis):
    
    lis_w, lis_t = to2lists(input)
    jam = Jamodealer(lis_w)
    lis_word, lis_tags, last_w = delete_end_only_test(jam.jamo, lis_t, lis_beta, lis)
    return lis_word, lis_tags, last_w

def prepro_after(input, lis, lis_w, lis_t):
    #lis_w, lis_t = to2lists(input)
    
    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    
    
    lis_w2, lis_t2, last_w = to2lists_beta(input, lis)
    print(lis_w2)
    lis_w2 = lis_w2[:-1]
    lis_t2 = lis_t2[:-1]
    
    
    
    
    union(space_location, lis_w2)
    #print(lis_w)
    union(space_location, lis_t2)
    union_t(lis_t2)
    union_w(lis_w2, lis_t2)
    
    
    
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w2)):
        str_w = str_w + lis_w2[i]
        str_t = str_t + lis_t2[i]
    
    
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag, ll = to2lists_beta(input, lis)
    
    w_last = lis_word[-1]
    t_last = lis_tag[-1]
    
    lis_word = lis_word[:-1]
    lis_tag = lis_tag[:-1]
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1
        
        #element = element[:-1]
        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
        
    lis_normal = []
    
    for i in data_w:
        jam_n = Jamodealer(i)
        lis_normal.append(jam_n.make_one())
    
    lis_target_ind = []
    word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
    for i in range(len(lis_normal)):
        if lis_normal[i]!=word_test_pre[i]:
            lis_target_ind.append(i)
    #data_w.append(w_last)
    
    return data_w, data_t, lis_ind, w_last, t_last, last_w, lis_normal, lis_target_ind

def convert_an(stc, tag, ex):
    
    ind_point = -1
    point = ''

            
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    
    if ex=='ㄹㅏ':
        return r_word+'ㅇㅛ'+r_pun
    elif ex=='ㄷㅏㄷㅓㄹㅏ':
        return r_word+'ㄷㅐㅇㅛ'+r_pun
    elif ex=='ㄷㅓㄹㅏ':
        wd = ''
        if 'ㄷㅏ' in r_word:
            print('dd')
            return r_word[:-1]+'ㅐㅇㅛ'+r_pun
        if r_word[-1] in jongsung_list:
            wd = r_word[-2]
        else:
            wd = r_word[-1]
        if wd=='ㅗ':
            return r_word+'ㅏㅇㅛ'+r_pun
        elif wd=='ㅜ':
            return r_word+'ㅓㅇㅛ'+r_pun
        elif wd=='ㅏ':
            return r_word+'ㅇㅏㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄴㅣ' or ex =='ㄴㅡㄴㄱㅏ' or ex =='ㅇㅡㄴㄱㅏ':
        return r_word+'ㄴㅏㅇㅛ'+r_pun
    elif ex=='ㅇㅑ':
        return r_word+'ㅇㅔㅇㅛ'+r_pun
#     elif ex=='ㄷㅓㄹㅏ':
#         if 'ㄷㅏ' in stc:
#             return stc[:-1]+'ㅔㅇㅛ'
#         return stc+'ㄷㅔㅇㅛ'
    elif ex=='ㅈㅣ':
        return r_word+'ㅈㅛ'+r_pun
    else:
        return r_word+ex+'ㅇㅛ'+r_pun
    
def convert_da(stc, tag):
#     print('come in convert da!')
#     print(stc)
#     print(tag)
    ind_point = -1
    point = ''
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
   
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    #print(r_word)
    jongsung = r_word[-1]
    if jongsung in jongsung_list:
        final = r_word +'ㅅㅡㅂㄴㅣㄷㅏ'+r_pun
    else: final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
    #print(final)
    return final

In [2]:
class Changer(object):
    
    def make_end_h(self, stc, tag, ex):
        #print(stc)
        flag = 1
        for i in H_LIST_jam:
            if ex.find(i) != -1:
                flag = 0
                break

        if flag==1:
            if 'ㄷㅏ'== ex[-2:]:
                return convert_da(stc, tag)
            else:
                return convert_an(stc, tag, ex)
        else:
            ind_point = -1
            point = ''
            for i in P_LIST:
                if stc.find(i) !=-1:
                    point = i
                    break
            if point in P_LIST:
                ind_point = stc.index(point)
    
            r_word = ''
            r_pun = ''
    
            if ind_point!=-1:
                r_word = stc[:ind_point]
                r_pun = stc[ind_point:]
            else:
                r_word = stc
            
            
            return r_word+ex+r_pun
    
    
    def to_high(self, input):
        result = input
        space_list = rememberSpace(input,' ')
        
        test_w, test_t = to2lists(result)
        pre_w, pre_t, pre_ind = prepro(result)
        
        lis_w, lis_t, lis_i, w_last, t_last, off_word, lis_normal, lis_target_ind = prepro_after(result, lis_end_2low)
        space_location = convertSpace(space_list, pre_w)
        lis_target_final = []
        
        for i in lis_target_ind:
            
            flag_ind = 0
            for h in H_LIST:
            
                if h == pre_w[i][-len(h):]:
                    flag_ind = 1
                    
            if flag_ind==0 and i not in lis_target_final:
                lis_target_final.append(i)
        
        flag = 0
        ind_tar = 0
        
        if len(lis_target_final)!=0:
            jam = Jamodealer(lis_normal)

            for i in range(len(lis_target_ind)):
                
                if lis_target_ind[i] not in lis_target_final:
                    jam.jamo[lis_target_ind[i]] = jam.jamo[lis_target_ind[i]]+off_word[i]
                else:
                    new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
                    jam.jamo[lis_target_ind[i]] = new_end
                
            union(space_location, jam.jamo)
            return jam.make_one()

        return input
    
    def to_high_test(self, input):
        # ...마침표 처리
        #'감사합니다.'
        result = input
        space_list = rememberSpace(input,' ')
        
        test_w, test_t = to2lists(result)
        pre_w, pre_t, pre_ind = prepro(result)
        #print(pre_w)
        
        lis_w, lis_t = to2lists(result)
        ending = lis_w[-2]
        for i in H_LIST:
            if ending.find(i) !=-1:
                print('Formal')
                return -1
        lis_w, lis_t, lis_i, w_last, t_last, off_word, lis_normal, lis_target_ind = prepro_after(result, lis_end_2low,lis_w, lis_t)
        #print(lis_normal)
        space_location = convertSpace(space_list, pre_w)
        lis_target_final = []
        
        if len(lis_target_ind)!=0:
            jam = Jamodealer(lis_normal)

            for i in range(len(lis_target_ind)):
                new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
                jam.jamo[lis_target_ind[i]] = new_end
            
            
            jam.jamo.append(w_last)
            union(space_location, jam.jamo)
            return jam.make_one()

        return input


    
        
    def processText(self,stc):
        ##여기서 하는 것이 제일 좋은데
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        
        num = 0
        while 1:
            if result[-1-num]!=' ':
                break
            else:
                num = num+1
        if num==0:
            rere = result
        else:
            rere = result[:-num]
        if rere[-1] =='?' or rere[-1] =='.' or rere[-1] =='!' or rere[-1] =='\"':
            rere = rere
        else:
            rere = rere+'.'
            flag = 1
        
        res = self.to_high_test(rere)
        if res ==-1:
            print("This is 존.댓.말.")
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res

In [8]:
ch = Changer()
ch.processText('그것을 하는 중 입니다.')

Formal
This is 존.댓.말.


-1

In [6]:
a = '그것을 하는 중 입니다.,"'
# ..같은 것을 잠시 다른 곳에 보관하고 없앤다.
lis_w, lis_t = to2lists(a)

ending = lis_w[-2]
for i in H_LIST:
    if ending.find(i) !=-1:
        print('Formal')

In [7]:
lis_t

['NP', 'JKO', 'VV', 'ETM', 'NNB', 'VCP+EF', 'SF', 'SY']

In [199]:
pun_tag = lis_t[-1]
pun_word = lis_w[-1]
del lis_t[-1]
lis_t
del lis_w[-1]
lis_w

['나', '는', '오늘', '밥', '을', '안', '먹', '습니다']

In [187]:
#lis_t = lis_t.replace(lis_t[-1], '')
lis_t.append(pun_tag)
lis_w.append(pun_word)

In [182]:
lis_w, lis_t

(['감사', '합니다', '.', '...'], ['NNG', 'XSV+EF', 'SF', 'SY'])

In [183]:
lis_t.count('SY') >=1 or lis_t.count('SF')>1 or lis_t.count('SC')>1, 

1

In [177]:
lis_t

['NNG', 'XSV+EF', 'SF']

In [9]:
from khaiii import KhaiiiApi

ModuleNotFoundError: No module named 'khaiii'